In [3]:
import pandas as pd
import ast
from tqdm import tqdm
import tweetnlp

#!pip install geopy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
# from ner_hf import *
import ast 
from scipy import stats
import matplotlib


import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
import re 
from nltk.corpus import stopwords
import string
cachedStopWords = stopwords.words("english")
cachedStopWords.extend(['california', 'san', 'sf', 'francisco', 'san', 'los', 'angeles', 'san', 'diego', 'homeless', 'homelessness', 'fran', 'person', 'people'])
from nltk.stem.snowball import SnowballStemmer
from collections import Counter
import re

import random 
from random import sample 

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 30}
plt.rc('font', **font)

stemmer = SnowballStemmer("english")

In [4]:
# load set of annotations

def load_anns(path_ner, path_flan_anns):
# def load_anns():
#     anns = pd.read_json('/home/brihi/characterizing-attitudes-towards-homelessness/original_twitter_data/ner_dump.jsonl', lines=True)
    anns = pd.read_json(path_ner, lines=True)

    print("read anns")
    print("starting to read flan anns")
#     flan_anns = pd.read_json("/home/brihi/characterizing-attitudes-towards-homelessness/original_twitter_data/all_3mil_flan_preds.jsonl", lines=True)
    flan_anns = pd.read_json(path_flan_anns, lines=True)

    print("read both anns and flan anns")
    flags = []

    for idx, row in flan_anns.iterrows():
        if idx % 10000 == 0:
            print(idx)
        finer = row['finer_updated']
        if len(finer) == 0 or '0' in finer or finer=='[]':
            flags.append(False)
        else:
            flags.append(True)
    flan_anns = flan_anns[flags]
    ret_anns = anns
    ret_anns['finer_updated'] = flan_anns.iloc[ret_anns.index]['finer_updated']


    print(ret_anns.shape)
    ret_anns = ret_anns.reset_index()
    ret_anns['idx'] = ret_anns.index
    return ret_anns


In [5]:
def weighted_log_odds_dirichlet(document1, document2, ngram=1, prior=.01, cv=None):
    # Referenced from: https://github.com/jmhessel/FightingWords/blob/master/fighting_words_py3.py 
    """
    Arguments:
    - document1, document2; a list of strings from each sample.
    - ngram; an int describing up to what n gram you want to consider (1 is unigrams,
        2 is bigrams + unigrams, etc). Ignored if a custom CountVectorizer is passed.
    - prior; either a float describing a uniform prior, or a vector describing a prior
        over vocabulary items. If a predefined vocabulary is being used, make sure to
        specify that when making your CountVectorizer object.
    - cv; an sklearn.feature_extraction.text.CountVectorizer object, if necessary.

    Returns:
    - A list of length equal to the amount of vocabulary, where each entry is an (n-gram, zscore) tuple.
    """
    if cv is None and type(prior) is not float:
        raise TypeError("If using a non-uniform prior, a CountVectorizer object with the "
                        "vocabulary parameter set needs to be passed as the cv argument.")
    if cv is None:
        cv = CountVectorizer(decode_error='ignore', min_df=10, max_df=.5, ngram_range=(1, ngram), binary=False,
                             max_features=15000)
    counts_mat = cv.fit_transform(document1 + document2).toarray()
    vocab_size = len(cv.vocabulary_)
    if type(prior) is float:
        priors = np.array([prior] * vocab_size)
    else:
        priors = prior
    z_scores = np.empty(np.array(priors).shape[0])
    count_matrix = np.empty([2, vocab_size], dtype=np.float32)
    count_matrix[0, :] = np.sum(counts_mat[:len(document1), :], axis=0)
    count_matrix[1, :] = np.sum(counts_mat[len(document1):, :], axis=0)
    a0 = np.sum(priors)
    n1 = 1. * np.sum(count_matrix[0, :])
    n2 = 1. * np.sum(count_matrix[1, :])
    for i in range(vocab_size):
        term1 = np.log((count_matrix[0, i] + priors[i]) / (n1 + a0 - count_matrix[0, i] - priors[i]))
        term2 = np.log((count_matrix[1, i] + priors[i]) / (n2 + a0 - count_matrix[1, i] - priors[i]))
        delta = term1 - term2
        var = 1. / (count_matrix[0, i] + priors[i]) + 1. / (count_matrix[1, i] + priors[i])
        z_scores[i] = delta / np.sqrt(var)
    index_to_term = {v: k for k, v in cv.vocabulary_.items()}
    sorted_indices = np.argsort(z_scores)
    out = []
    for i in sorted_indices:
        out.append((index_to_term[i], z_scores[i]))

    return out


def count_tokens_in_tweets(tweets):
    # Combine all tweets into a single string
    combined_tweets = " ".join(tweets)
    
    # Use regular expressions to find tokens
    # This regex considers words, punctuation, and other significant tokens
    tokens = re.findall(r'\S+', combined_tweets)
    
    # Count the occurrences of each token
    token_counts = Counter(tokens)
    
    return token_counts


def stop_word_removal(text):
    text = ' '.join([word for word in text.split() if word not in cachedStopWords])
    return text
    
    
def stem(text):
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text


def preproc(text):
    translator = str.maketrans('', '', string.punctuation)

    processed_text = []
    for twet in tqdm(text):
        val = re.sub('<[^>]+>', '', twet)
        val2 = re.sub('@[^\s]+','',val)
        val3 = val2.translate(translator)
        val4 = stop_word_removal(val3)
        val5 = stem(val4)

        processed_text.append(val5)
    return processed_text

# Load 2.4M Analysis Set

In [ ]:
# anns = load_anns()
path_ner = ''
path_flan = ''
anns = load_anns(path_ner, path_flan)

df = anns

# Segment out California posts 

In [7]:
model = tweetnlp.load_model('sentiment')
df_cali = df[df['California'] == 1]

sent_pos = []
sent_neg = []
sent_neut = []
sent_neg_combined = []

for idx, row in df_cali.iterrows():
    print(idx)
    pos = model.sentiment(row['text'], return_probability=True)['probability']['positive']
    neg =  model.sentiment(row['text'], return_probability=True)['probability']['negative']
    net =  model.sentiment(row['text'], return_probability=True)['probability']['neutral']
    com = neg+net
    sent_pos.append(pos)
    sent_neg.append(neg)
    sent_neut.append(net)
    sent_neg_combined.append(com)
    
df_cali['sentiment_positive'] = sent_pos
df_cali['sentiment_negative'] = sent_neg
df_cali['sentiment_neutral'] = sent_neut
df_cali['sentiment_combined'] = sent_neg_combined
# df_cali.to_csv('../regionality/df_cali_sentiment.csv')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/jaspreet/.conda/envs/lxmert/lib/python3.9/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11000). Please update your GPU driver by downloading and installing a new version from the URL: http:/

28
43
87
100
106
108
169
205
290
332
347
354
374
378
387
404
409
432
440
512
517
523
560
568
584
594
645
723
728
753
767
790
809
810
820
858
861
878
879
895
910
957
993
1019
1040
1041
1043
1055
1059
1081
1117
1167
1172
1181
1182
1220
1229
1253
1297
1298
1313
1389
1397
1433
1440
1455
1456
1530
1586
1638
1660
1677
1718
1733
1743
1746
1779
1789
1859
1975
1981
2041
2052
2075
2080
2121
2186
2229
2243
2301
2310
2314
2343
2427
2457
2483
2484
2514
2585
2616
2644
2645
2648
2664
2690
2736
2798
2840
2843
2946
2962
2984
3008
3017
3033
3045
3135
3137
3140
3180
3181
3222
3229
3247
3268
3289
3295
3306
3334
3354
3369
3392
3400
3417
3418
3434
3494
3498
3572
3577
3585
3603
3616
3643
3663
3687
3690
3691
3698
3702
3722
3731
3738
3740
3747
3782
3788
3797
3801
3805
3811
3816
3821
3822
3834
3840
3845
3852
3859
3870
3871
3883
3892
3900
3915
3926
3934
3964
3977
3987
3988
3989
3991
3998
4002
4012
4018
4036
4060
4067
4071
4080
4085
4087
4093
4124
4139
4142
4145
4146
4161
4162
4165
4166
4170
4177
4270
4285
4286
4

/tmp/ipykernel_71499/3961464899.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cali['sentiment_positive'] = sent_pos
/tmp/ipykernel_71499/3961464899.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cali['sentiment_negative'] = sent_neg
/tmp/ipykernel_71499/3961464899.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [9]:
df_all_text = df_cali['tweet_processed'].tolist()
df_processed = preproc(df_all_text)
df_cali['tweet_processed'] = df_processed

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 5373/5373 [00:01<00:00, 4760.51it/s]
/tmp/ipykernel_71499/2596996161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cali['tweet_processed'] = df_processed


# Positive / negative sentiment

In [10]:
cali_sentiment = df_cali
# cali_sentiment = pd.read_csv('df_cali_sentiment.csv')
pos_tweets = []
neg_tweets = []

for idx, row in cali_sentiment.iterrows():
    if row['sentiment_positive'] > 0.5:
        pos_tweets.append(row['tweet_processed'])
    elif row['sentiment_negative'] > 0.5:
        neg_tweets.append(row['tweet_processed'])

In [11]:
random.seed(10)
neg_sample = list(set(sample(neg_tweets,400)))


In [12]:
combined = pos_tweets+neg_sample
word_count = count_tokens_in_tweets(combined)

all_counts = {k: 1 / v for k, v in word_count.items()}


prior = []
cv_vocab = {}
for w, p in all_counts.items():
    prior.append(p)
    cv_vocab[w] = len(prior) - 1
    
informed_out = weighted_log_odds_dirichlet(pos_tweets, neg_sample, ngram=2, prior=prior, cv=CountVectorizer(vocabulary=cv_vocab))
print(dict(informed_out))

{'drug': -3.837343131343643, 'go': -2.911839265708185, 'crime': -2.7421422753123803, 'addict': -2.6139642794231555, 'rate': -2.4943587097620403, 'high': -2.4930500773531765, 'get': -2.4502709072881794, 'highest': -2.337263096417089, 'run': -2.3281612110974965, 'think': -2.3265682476477836, 'popul': -2.2988479917498896, 'countri': -2.2368088124473315, 'democrat': -2.2368088124473315, 'happen': -2.190461832079439, 'open': -2.163354992731075, 'mental': -2.1172284633621037, 'like': -1.991975844309105, 'bad': -1.9667251965930765, 'portland': -1.9529136463542491, 'seem': -1.9529136463542491, 'anyth': -1.9529136463542491, 'man': -1.9326720138868902, 'street': -1.9260251454563435, 'noth': -1.8597542637210773, 'million': -1.8579507362393464, 'leav': -1.8579507362393464, 'fece': -1.8579507362393464, 'pay': -1.8579507362393464, 'number': -1.7573910019113426, 'big': -1.7573910019113426, 'control': -1.7573910019113426, 'even': -1.7482756431612527, 'ill': -1.7408384661189835, 'thousand': -1.74083846

In [ ]:
cali_pos_neg_df = pd.DataFrame(informed_out)
# cali_pos_neg_df.to_csv('cali_sentiment_logodds_informed.csv')

# Soln. / not Soln. 

In [13]:
df_all_text = df['tweet_processed'].tolist()
df_processed = preproc(df_all_text)
df['tweet_processed'] = df_processed

df_cali = df[df['California'] == 1]
idx_solutions_interventions = []
idx_not_solutions = []

for idx, row in df_cali.iterrows():
    finer = row['finer_updated']
    if type(finer)==list:
        if 'solutions_interventions' in finer:
            idx_solutions_interventions.append(row['idx'])
        else:
            idx_not_solutions.append(row['idx'])

cali_solutions_interventions = df.loc[idx_solutions_interventions]
cali_not_solutions_interventions = df.loc[idx_not_solutions]


100%|███████████████████████████████████████████████████████████████████████████████████████████| 147456/147456 [00:26<00:00, 5624.60it/s]


In [14]:
random.seed(10)
temp = list(set(cali_not_solutions_interventions['tweet_processed'].tolist()))
cali_not_solutions_interventions_sample = sample(temp,2000)
cali_soln_interventions_list = list(set(cali_solutions_interventions['tweet_processed'].tolist()))


In [15]:

combined = cali_soln_interventions_list+cali_not_solutions_interventions_sample
word_count = count_tokens_in_tweets(combined)
all_counts = {k: 1 / v for k, v in word_count.items()}

prior = []
cv_vocab = {}
for w, p in all_counts.items():
    prior.append(p)
    cv_vocab[w] = len(prior) - 1
    
informed_out = weighted_log_odds_dirichlet(cali_soln_interventions_list, cali_not_solutions_interventions_sample, ngram=2, prior=prior, cv=CountVectorizer(vocabulary=cv_vocab))
print(dict(informed_out))

{'crime': -7.371715779993168, 'drug': -6.704378014008433, 'state': -5.80893182915895, 'illeg': -5.720025849075074, 'shit': -5.237517892775874, 'addict': -4.993786921560901, 'democrat': -4.861038844943392, 'left': -4.777427693271276, 'man': -4.634363588973535, 'billion': -4.317919880908862, 'street': -4.275487279308039, 'walk': -4.139643803709699, 'look': -4.081170346751915, 'newsom': -4.057337489158792, 'highest': -3.9873843278648744, 'tax': -3.9809814290491436, 'leav': -3.881794800421167, 'liber': -3.87057109589357, 'like': -3.8525565852522137, 'white': -3.8515317375928992, 'high': -3.7720731389726168, 'governor': -3.7647542337558315, 'run': -3.637859888220318, 'trash': -3.501099134584815, 'seen': -3.4655351070244884, 'guy': -3.424660829353116, 'dude': -3.41639071662484, 'needl': -3.399793264265403, 'encamp': -3.3111072345573933, 'worst': -3.307885919569529, 'million': -3.2939583663659073, 'everywher': -3.2870550279604815, 'polici': -3.284994901286242, 'sidewalk': -3.264715847192359, 

In [ ]:
cali_soln_not_soln_df = pd.DataFrame(informed_out)
# cali_soln_not_soln_df.to_csv('cali_soln_not_soln_informed.csv')